In [26]:
# assim que abriu o shell da quit que ele fecha
%run django_shell_to_jupter.py

Python 3.7.9 (default, Feb  1 2024, 17:42:47) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.34.0 -- An enhanced Interactive Python. Type '?' for help.




In [27]:
def modify_cat(json_obj):
    if isinstance(json_obj, dict):
        if "cat" in json_obj:
            cat_value = json_obj["cat"]
            if isinstance(cat_value, list):
                for i in range(len(cat_value)):
                    if isinstance(cat_value[i], dict) and "var" in cat_value[i]:
                        var_value = cat_value[i]["var"]
                        if isinstance(var_value, str) and var_value.startswith("index."):
                            cat_value[i] = "." + var_value.split(".")[1]

                    if isinstance(cat_value[i], str) and str(cat_value[i]).startswith("data.form_data."):
                        cat_value[i] = str(cat_value[i]).replace("data.form_data.", "")
        return {key: modify_cat(value) for key, value in json_obj.items()}
    elif isinstance(json_obj, list):
        return [modify_cat(item) for item in json_obj]
    else:
        return json_obj

In [28]:
from helpers.strings import to_snake_case


def apply_to_snake_case(obj):
    if isinstance(obj, dict):
        return {
            apply_to_snake_case(key): apply_to_snake_case(value)
            for key, value in obj.items()
        }
    elif isinstance(obj, list):
        return [apply_to_snake_case(element) for element in obj]
    elif isinstance(obj, str):
        return to_snake_case(obj)
    else:
        return obj

In [29]:
from json_logic import jsonLogic


def valid_if(form_data: dict, json_logic: dict) -> bool:
    json_logic = apply_to_snake_case(json_logic)
    modify_cat(json_logic)

    return jsonLogic(json_logic, form_data)

In [30]:
from typing import List


def get_field_by_members(members, form_fields) -> List[dict]:
    fields = form_fields.get("fields", [])
    return [field for field in fields if field["id"] in members]

In [31]:
from apps.reportings.models import Reporting

# reportings = Reporting.objects.filter(
#     # pk="0532bed2-86eb-4e74-ba3e-93f52eeb2d42"
# ).first()
reportings = Reporting.objects.filter(occurrence_type__isnull=False).first()

occurrence_type = reportings.occurrence_type

In [32]:
form_fields = occurrence_type.form_fields
form_data = reportings.form_data

In [33]:
from helpers.strings import deep_keys_to_snake_case


data_new = []
for group in form_fields.get("groups", []):
    members = group.get("members", [])
    _data = {"label": group.get("display_name"), "fields_id": members, "fields": []}
    fields = get_field_by_members(members, form_fields)
    fields = deep_keys_to_snake_case(fields)
    for field in fields:
        api_name = to_snake_case(field.get("api_name"))
        data_type_field = field.get("data_type")
        print_if_field = field.get("print_if")
        display_if_field = field.get("display_if")
        _data["data_type"] = field.get("data_type")
        if data_type_field in ["string", "boolean", "timestamp", "textArea"]:
            if (print_if_field is None and display_if_field is None) or (print_if_field and (valid_if(form_data, print_if_field) or valid_if(form_data, print_if_field) is None))  or (display_if_field and (valid_if(form_data, display_if_field) or valid_if(form_data, display_if_field) is None)):
                _fields = {"label": field.get("display_name", "", ), "value": form_data.get(api_name), "data_type": field.get("data_type")}
                _data["fields"].append(_fields)

        elif data_type_field == "arrayOfObjects":
            _data["data_type"] = field.get("data_type")
            _data["fields"] = []
            blocks = []
            inner_fields = field.get("inner_fields", [])
            for inner in inner_fields:
                inner_name = to_snake_case(inner.get("api_name"))
                print_if_inner = inner.get("print_if")
                display_if_inner = inner.get("display_if")
                if print_if_inner and (valid_if(form_data, print_if_inner) or valid_if(form_data, print_if_inner) is None) or (print_if_inner is None and display_if_inner is None):

                    for form_data_field in form_data.get(api_name, {}):
                        for x in inner.get("select_options", {}).get("options", []):
                            _v = form_data_field.get(inner_name)
                            if x.get("value") == _v:
                                _values = x.get("name")
                                blocks.append({"label": inner.get("display_name"), "values":_values, "data_type": inner.get("data_type")})
                                break
                elif display_if_inner and (valid_if(form_data, display_if_inner) or valid_if(form_data, display_if_inner) is None) or (print_if_field is None and display_if_inner is None):
                    for form_data_field in form_data.get(api_name, {}):
                        for x in inner.get("select_options", {}).get("options", []):
                            _v = form_data_field.get(inner_name)
                            if x.get("value") == _v:
                                _values = x.get("name")
                                blocks.append({"label": inner.get("display_name"), "values":_values, "data_type": inner.get("data_type")})
                                break

            _data["fields"].append(blocks)

        else:
            print(field)

    if _data not in data_new:
        data_new.append(_data)


print(data_new)

ImportError: cannot import name 'deep_keys_to_snake_case' from 'helpers.strings' (/home/leograciano/Projects/Kartado/kartado-backend/helpers/strings.py)

In [ ]:
company = reportings.company

custom_options = company.custom_options

print(custom_options)